In [0]:
%sql
CREATE TABLE IF NOT EXISTS ankurnayyar_cat1.default.central_constraints (
    flow_name STRING,
    table_name STRING,
    constraint_name STRING,
    constraint_expression STRING,
    added_at TIMESTAMP,
    enabled BOOLEAN
)
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
);


In [0]:
%sql
INSERT INTO ankurnayyar_cat1.default.central_constraints (flow_name, table_name, constraint_name, constraint_expression, enabled)
VALUES
 ('silver_events_patient_data_cdc_scd2', 'silver_events_patient_data_scd2', 'id_not_null', 'ID IS NOT NULL', TRUE),
 ('silver_events_patient_data_cdc_scd2', 'silver_events_patient_data_scd2', 'businessid_not_null', 'BusinessID IS NOT NULL', TRUE),
 ('silver_events_patient_data_cdc_scd2', 'silver_events_patient_data_scd2', 'dcreated_not_null', 'D_created IS NOT NULL', TRUE),
 ('silver_events_patient_data_cdc_scd2', 'silver_events_patient_data_scd2', 'state_two_chars', "D_state RLIKE '^[A-Z]{2}$'", TRUE),
 ('silver_events_patient_data_cdc_scd2', 'silver_events_patient_data_scd2', 'zipcode_us_pattern', "D_zipCode RLIKE '^[0-9]{5}(-[0-9]{4})?$'", TRUE);


In [0]:
%sql
SELECT *
FROM ankurnayyar_cat1.default.central_constraints
WHERE flow_name = 'silver_events_patient_data_cdc_scd2'
  AND table_name = 'silver_events_patient_data_scd2'
  AND enabled = TRUE;


In [0]:
%sql
-- Dynamically filter data based on central constraints
CREATE OR REPLACE TEMP VIEW patient_cleaned AS
SELECT *
FROM silver_events_patient_data_scd2
WHERE ID IS NOT NULL
  AND BusinessID IS NOT NULL
  AND D_created IS NOT NULL
  AND D_state RLIKE '^[A-Z]{2}$'
  AND D_ZipCode RLIKE '^[0-9]{5}(-[0-9]{4})?$';


In [0]:
%sql
-- Collect all enabled constraints as a SQL expression
WITH rules AS (
  SELECT concat_ws(' AND ', collect_list(constraint_expression)) AS where_clause
  FROM central_constraints
  WHERE flow_name = 'silver_events_patient_data_cdc_scd2'
    AND table_name = 'silver_events_patient_data_scd2'
    AND enabled = TRUE
)
SELECT sql_text
FROM (
  SELECT 'SELECT * FROM silver_events_patient_data_scd2 WHERE ' || where_clause AS sql_text
  FROM rules
);


In [0]:
%sql
CREATE TABLE IF NOT EXISTS ankurnayyar_cat1.default.silver_events_patient_data_quarantine
(
  flow_name STRING,
  table_name STRING,
  constraint_name STRING,
  constraint_expression STRING,
  violated_at TIMESTAMP,
  record STRUCT<>
)
TBLPROPERTIES ('delta.enableChangeDataFeed'='true');
